# Pull Repo

In [1]:
import os
# os.environ['GITHUB_TOKEN'] = ''
github_token = os.getenv("GITHUB_TOKEN")

In [4]:
import nest_asyncio
nest_asyncio.apply()

In [197]:
from llama_index.readers.github import GithubRepositoryReader, GithubClient

client = github_client = GithubClient(github_token=github_token, verbose=False)

reader = GithubRepositoryReader(
    github_client=github_client,
    owner="Infineon",
    repo="mtb-example-btsdk-low-power-20819",
    use_parser=False,
    verbose=True,
    # filter_directories=(
    #     ["docs"],
    #     GithubRepositoryReader.FilterType.INCLUDE,
    # ),
    filter_file_extensions=(
        [
            ".png",
            ".jpg",
            ".jpeg",
            ".gif",
            ".svg",
            ".ico",
            "json",
            ".ipynb",
        ],
        GithubRepositoryReader.FilterType.EXCLUDE,
    ),
)

documents = reader.load_data(branch="master")

current path: 
tree data: GitTreeResponseModel(sha='1544236817d8d39bc023530d5f232efd93040b20', url='https://api.github.com/repos/Infineon/mtb-example-btsdk-low-power-20819/git/trees/1544236817d8d39bc023530d5f232efd93040b20', tree=[GitTreeResponseModel.GitTreeObject(path='.gitignore', mode='100644', type='blob', sha='a053eade2a77ebf0c1e34d65621041247e21af43', url='https://api.github.com/repos/Infineon/mtb-example-btsdk-low-power-20819/git/blobs/a053eade2a77ebf0c1e34d65621041247e21af43', size=2678), GitTreeResponseModel.GitTreeObject(path='LICENSE', mode='100644', type='blob', sha='01d5e5857d76f6eeb507c5e33e2f8b46cf1b4e94', url='https://api.github.com/repos/Infineon/mtb-example-btsdk-low-power-20819/git/blobs/01d5e5857d76f6eeb507c5e33e2f8b46cf1b4e94', size=12551), GitTreeResponseModel.GitTreeObject(path='Makefile', mode='100644', type='blob', sha='25f8678560e357b958ee66db00197b3c21972f91', url='https://api.github.com/repos/Infineon/mtb-example-btsdk-low-power-20819/git/blobs/25f8678560e3

In [198]:
for doc in documents:
    doc.metadata['repo']='mtb-example-btsdk-low-power-20819'
    doc.metadata['folder']= doc.metadata['repo'] + '/' + doc.metadata['file_path'].split('/')[0] if '/' in doc.metadata['file_path'] else ''
    print(doc.metadata)

{'file_path': '.gitignore', 'file_name': '.gitignore', 'url': 'https://github.com/Infineon/mtb-example-btsdk-low-power-20819/blob/master/.gitignore', 'repo': 'mtb-example-btsdk-low-power-20819', 'folder': ''}
{'file_path': 'LICENSE', 'file_name': 'LICENSE', 'url': 'https://github.com/Infineon/mtb-example-btsdk-low-power-20819/blob/master/LICENSE', 'repo': 'mtb-example-btsdk-low-power-20819', 'folder': ''}
{'file_path': 'Makefile', 'file_name': 'Makefile', 'url': 'https://github.com/Infineon/mtb-example-btsdk-low-power-20819/blob/master/Makefile', 'repo': 'mtb-example-btsdk-low-power-20819', 'folder': ''}
{'file_path': 'README.md', 'file_name': 'README.md', 'url': 'https://github.com/Infineon/mtb-example-btsdk-low-power-20819/blob/master/README.md', 'repo': 'mtb-example-btsdk-low-power-20819', 'folder': ''}
{'file_path': 'app_bt_cfg.c', 'file_name': 'app_bt_cfg.c', 'url': 'https://github.com/Infineon/mtb-example-btsdk-low-power-20819/blob/master/app_bt_cfg.c', 'repo': 'mtb-example-btsdk

# Pinecone to store db

In [30]:
from pinecone import Pinecone, ServerlessSpec

# os.environ["PINECONE_API_KEY"] = ''
os.environ["PINECONE_API_KEY"] = ''

In [31]:
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

In [32]:
os.environ['MISTRAL_API_KEY'] = ''

In [33]:
from llama_index.embeddings.mistralai import MistralAIEmbedding

model_name="mistral-embed"

embed_model = MistralAIEmbedding(model_name=model_name)

In [34]:
from llama_index.core import Settings

Settings.embed_model = embed_model

In [35]:
index_name = "llamaindex-ragathon-mohit-full-docs"

pc.create_index(
    name=index_name,
    dimension=1024,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [36]:
pinecone_index = pc.Index(index_name)

# upload docs to index

In [37]:
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.pinecone import PineconeVectorStore

In [44]:
import asyncio

async def create_index_async(documents):
    vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    
    # Use asyncio.gather() to handle multiple document insertions
    index = await VectorStoreIndex.from_documents(documents, storage_context=storage_context)
    return index

In [47]:
import getpass
import os
import time

from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

In [199]:
import time

index_name = "langchain-test-index-mohit-full-docs"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [50]:
os.environ['OPENAI_API_KEY'] = ''
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [200]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [161]:
from uuid import uuid4

# vector_store.add_documents(documents=documents, ids=uuids)

In [204]:
# Assuming 'documents' is your list of Document instances
from langchain_core.documents import Document

updated_documents = []

for document in documents:
    # Create a new Document instance with the updated attributes
    updated_document = Document(
        metadata=document.metadata,  # Keep metadata the same
        page_content=document.text,  # Change text to page_content
    )
    updated_documents.append(updated_document)

# updated_documents now contains the modified Document instances


In [205]:
len(updated_documents)

11

In [206]:
uuids = [str(uuid4()) for doc in updated_documents]

# Chunking

In [165]:
from langchain_text_splitters import TokenTextSplitter
chunks=[]
for doc in updated_documents:

    text_splitter = TokenTextSplitter(chunk_size=400, chunk_overlap=80)
    content = text_splitter.split_text(doc.page_content)
    for chunk in content:
        chunks.append(
            Document(
                metadata=doc.metadata,
                page_content = chunk
            ),
            
        )

In [ ]:
chunks

In [171]:
uuids = [str(uuid4()) for doc in chunks]

In [ ]:
for i in chunks:
    print(i.metadata)

In [207]:
vector_store.add_documents(documents=chunks, ids=uuids)

['e514d60e-0890-4ac4-afe9-aeb8476130e1',
 '9a5c805f-7144-40c8-9753-81ac70bed475',
 'd04a7ff8-f591-4308-b4a7-720efad3dcd3',
 '40e59566-fdcf-42fd-9b97-699d1b73aaac',
 'a0eeec9a-3318-4578-92ac-fc74dd43b6dc',
 '743f2148-28f7-421f-a619-756bf7dff12b',
 '7a6db576-6d2a-4e76-924e-cc7dc7a5ed48',
 '69127ea0-a3ab-4fce-8f0a-3c6ac12c4de3',
 '11e22a62-f15b-44d5-a398-ee7f657bd8e4',
 '3274aa90-5785-4a08-ba3b-46ca3732be46',
 'c86e4ea2-48a3-4efa-b2ef-034ab374c914']

In [105]:
results = vector_store.similarity_search_with_relevance_scores(
    '''Group: Home/Forums/WirelessConnectivity/AIROC Bluetooth
Topic: cybt-213043-02 not entering sleep mode
Hi,
I am using cybt-213043-02 eval board. I am running https://github.com/Infineon/mtb-example-btsdk-low-power-20819 example with a tweak that changes the eval board to cybt-213043-eval.
I can see the output on the PUART when I press the user button and the ble works ok.
However the board never enters sleep mode. I see a constant ~2.5mA usage and the sleep permit handler is never called. Also sleep callback is never called.
The HOST_WAKE and DEV_WAKE pins are floating. I have tried tying them to either VCC or GND without effect. Is there some additional configuration or hw requirement to trigger sleep mode.
Thank you for response.''',
    k=15,
)

In [ ]:
s = vector_store.similarity_search('How to deploy an edge in AWS', k=4)
s

In [106]:
context=[]
for i in results:
    context.append(i[0].page_content)
    print(i[0].metadata)

{'file_name': 'README.md', 'file_path': 'README.md', 'url': 'https://github.com/Infineon/mtb-example-btsdk-low-power-20819/blob/master/README.md'}
{'file_name': 'README.md', 'file_path': 'README.md', 'url': 'https://github.com/Infineon/mtb-example-btsdk-low-power-20819/blob/master/README.md'}
{'file_name': 'low_power_20819_ble.c', 'file_path': 'low_power_20819_ble.c', 'url': 'https://github.com/Infineon/mtb-example-btsdk-low-power-20819/blob/master/low_power_20819_ble.c'}
{'file_name': 'README.md', 'file_path': 'README.md', 'url': 'https://github.com/Infineon/mtb-example-btsdk-low-power-20819/blob/master/README.md'}
{'file_name': 'low_power_20819.c', 'file_path': 'low_power_20819.c', 'url': 'https://github.com/Infineon/mtb-example-btsdk-low-power-20819/blob/master/low_power_20819.c'}
{'file_name': 'README.md', 'file_path': 'README.md', 'url': 'https://github.com/Infineon/mtb-example-btsdk-low-power-20819/blob/master/README.md'}
{'file_name': 'low_power_20819.c', 'file_path': 'low_power

In [130]:
type(vector_store)

langchain_pinecone.vectorstores.PineconeVectorStore

In [ ]:
vector_store.delete(ids=uuids)

In [84]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model='gpt-4o',
    temperature=0.1,
    max_tokens=2048
)

In [87]:
from langchain.prompts import PromptTemplate
prompt_template = """
Human: You are an expert at generating answers from the context given.

You are given a list of documents as context to use as reference while generating the answer.

Use the documents as reference. if you dont know say you dont know.

Relevant Documents:

<context>
{context}
</context>

<question>
{query}
</question>

Assistant:"""

doc_prompt = PromptTemplate(template=prompt_template, 
                               input_variables=["context","query"])

In [93]:
low_power_as_context = '''/*******************************************************************************
* Copyright 2020-2022, Cypress Semiconductor Corporation (an Infineon company) or
* an affiliate of Cypress Semiconductor Corporation.  All rights reserved.
*
* This software, including source code, documentation and related
* materials ("Software") is owned by Cypress Semiconductor Corporation
* or one of its affiliates ("Cypress") and is protected by and subject to
* worldwide patent protection (United States and foreign),
* United States copyright laws and international treaty provisions.
* Therefore, you may use this Software only as provided in the license
* agreement accompanying the software package from which you
* obtained this Software ("EULA").
* If no EULA applies, Cypress hereby grants you a personal, non-exclusive,
* non-transferable license to copy, modify, and compile the Software
* source code solely for use in connection with Cypress's
* integrated circuit products.  Any reproduction, modification, translation,
* compilation, or representation of this Software except as specified
* above is prohibited without the express written permission of Cypress.
*
* Disclaimer: THIS SOFTWARE IS PROVIDED AS-IS, WITH NO WARRANTY OF ANY KIND,
* EXPRESS OR IMPLIED, INCLUDING, BUT NOT LIMITED TO, NONINFRINGEMENT, IMPLIED
* WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE. Cypress
* reserves the right to make changes to the Software without notice. Cypress
* does not assume any liability arising out of the application or use of the
* Software or any product or circuit described in the Software. Cypress does
* not authorize its products for use in any products where a malfunction or
* failure of the Cypress product may reasonably be expected to result in
* significant property damage, injury or death ("High Risk Product"). By
* including Cypress's product in a High Risk Product, the manufacturer
* of such system or application assumes all risk of such use and in doing
* so agrees to indemnify Cypress against all liability.
*******************************************************************************/

/*******************************************************************************
 * File name: low_power_20819.c
 *
 * Description: Main file which is the entry point to the application.
 *              The application_start function initializes UART for trace
 *              messages, initializes the BT stack and configures sleep
 *              parameters. The low_power_sleep_handler function handles the
 *              sleep related callbacks from the PMU.
 *              Refer to the CE document CE225540_CYW20819_Low_Power.pdf
 *              provided along with the code example for more details.
 *
 * Features demonstrated:
 *              ePDS and HID-Off functionality with BLE advertisements (ADV) and
 *              connection.
 *
 *
 *  Controls:
 *  - Start ADV with button press when not connected.
 *  - Disconnect and enter HID-Off with button press when connected.
 *  - Wake from HID-Off on button press.
 *
 *  Note: Do not connect HCI UART when running this application as it will
 *  prevent the device from entering sleep as we have configured sleep mode as
 *  WICED_SLEEP_MODE_NO_TRANSPORT (see header file wiced_sleep.h)
 ******************************************************************************/

#include "wiced.h"
#include "wiced_bt_dev.h"
#include "wiced_hal_gpio.h"
#include "wiced_platform.h"
#include "wiced_bt_trace.h"
#include "wiced_bt_stack.h"
#include "app_bt_cfg.h"
#include "wiced_sleep.h"
#include "wiced_rtc.h"
#include "wiced_hal_mia.h"
#include "GeneratedSource/cycfg_pins.h"
#include "low_power_20819.h"

/*******************************************************************************
 *                    Macros
 ******************************************************************************/
/* Pass 10000000 to HID-Off API to sleep for 10 seconds */
#define HIDOFF_SLEEP_TIME 10000000

/*******************************************************************************
 * Variable Definitions
 ******************************************************************************/

/* sleep configuration */
wiced_sleep_config_t    low_power_sleep_config;

/*******************************************************************************
 *                  Function Prototypes
 ******************************************************************************/
static uint32_t               low_power_sleep_handler(wiced_sleep_poll_type_t type);
static void                   low_power_post_sleep_cb(
                              wiced_bool_t restore_configuration);

extern wiced_bt_dev_status_t  low_power_20819_bt_management_callback(
                              wiced_bt_management_evt_t event,
                              wiced_bt_management_evt_data_t *p_event_data);

/******************************************************************************/

/*******************************************************************************
 * Function Definitions
 ******************************************************************************/

/*******************************************************************************
 * Function Name: application_start()
 *******************************************************************************
* Summary: Initialize transport configuration, register BLE
*          management event callback and configure sleep.
*
* Parameters:
*   None
*
* Return:
*  None
*
*******************************************************************************/
void application_start(void)
{
    /* Following line of code is required to enable Hibernate */
    g_foundation_config_PMUflags |= PMU_CONFIG_FLAGS_ENABLE_SDS;

    wiced_sleep_wake_reason_t wake_reason = 0;

    /* Set Debug UART as WICED_ROUTE_DEBUG_TO_PUART to see debug traces on
     * Peripheral UART (PUART) */
    wiced_set_debug_uart(WICED_ROUTE_DEBUG_TO_PUART);

    WICED_BT_TRACE ("\r\n==========================================================\r\n\n"
                    "                  Low Power 20819\r\n\r\n"
                    "==========================================================\r\n"
                    "  This application implements low power modes (ePDS and\r\n"
                    "  HID-Off) in CYW20819\r\n"
                    "==========================================================\r\n\n");

    wake_reason = wiced_sleep_hid_off_wake_reason();

    if(WICED_SLEEP_WAKE_REASON_POR == wake_reason)
    {
        WICED_BT_TRACE("Reset due to POR\n");
    }
    else if(WICED_SLEEP_WAKE_REASON_HIDOFF_TIMEOUT == wake_reason)
    {
        WICED_BT_TRACE("Reset due to timed wake from HID-Off\n");
    }
    else
    {
        if(wiced_hal_gpio_get_pin_interrupt_status(
           WICED_GET_PIN_FOR_BUTTON(WICED_PLATFORM_BUTTON_1)))
        {
            WICED_BT_TRACE("Reset due to GPIO wake from HID-Off\n");
        }
        else
        {
            WICED_BT_TRACE("Reset due to other reasons\n");
        }
    }

    /* Initialize Bluetooth Controller and Host Stack */
    if(WICED_BT_SUCCESS != wiced_bt_stack_init(
                           low_power_20819_bt_management_callback,
                           &wiced_bt_cfg_settings,
                           wiced_bt_cfg_buf_pools))
    {
        WICED_BT_TRACE("Bluetooth LE Stack initialization failed\r\n");
    }

    /* configure to sleep if sensor is idle */
    low_power_sleep_config.sleep_mode               = WICED_SLEEP_MODE_NO_TRANSPORT;
    low_power_sleep_config.device_wake_mode         = WICED_SLEEP_WAKE_ACTIVE_LOW;
    low_power_sleep_config.device_wake_source       = WICED_SLEEP_WAKE_SOURCE_GPIO;
    low_power_sleep_config.device_wake_gpio_num     = WICED_GET_PIN_FOR_BUTTON(WICED_PLATFORM_BUTTON_1);
    low_power_sleep_config.host_wake_mode           = WICED_SLEEP_WAKE_ACTIVE_HIGH;
    low_power_sleep_config.sleep_permit_handler     = low_power_sleep_handler;
    low_power_sleep_config.post_sleep_cback_handler = low_power_post_sleep_cb;

    if(WICED_BT_SUCCESS != wiced_sleep_configure(&low_power_sleep_config))
    {
        WICED_BT_TRACE("Sleep Configure failed\r\n");
    }
}

/*******************************************************************************
 * Function Name: uint32_t low_power_sleep_handler(wiced_sleep_poll_type_t type)
 *******************************************************************************
 * Summary: Callback for sleep permissions.
 *
 * Parameters:
 *   wiced_sleep_poll_type_t type          : Poll type (see #wiced_sleep_poll_type_t)
 *
 * Return:
 *   uint32_t: if type == WICED_SLEEP_POLL_SLEEP_PERMISSION, application should
 *             return WICED_SLEEP_ALLOWED_WITHOUT_SHUTDOWN or
 *             WICED_SLEEP_NOT_ALLOWED. If type == WICED_SLEEP_POLL_TIME_TO_SLEEP,
 *              application should return WICED_SLEEP_MAX_TIME_TO_SLEEP
 *
 ******************************************************************************/
uint32_t low_power_sleep_handler(wiced_sleep_poll_type_t type)
{
    uint32_t ret = WICED_SLEEP_NOT_ALLOWED;

    switch(type)
    {
        case WICED_SLEEP_POLL_SLEEP_PERMISSION:
            if(HIDOFF == low_power_20819_current_state)
            {
                /* This allows the device to enter HID-Off */
                ret = WICED_SLEEP_ALLOWED_WITH_SHUTDOWN;

            }
            else
            {
                /* This allows the device to enter ePDS */
                ret = WICED_SLEEP_ALLOWED_WITHOUT_SHUTDOWN;

            }
            break;
        case WICED_SLEEP_POLL_TIME_TO_SLEEP:
            if(HIDOFF == low_power_20819_current_state)
            {
                ret = HIDOFF_SLEEP_TIME;
            }
            else
            {
                ret = WICED_SLEEP_MAX_TIME_TO_SLEEP;
            }
            break;
    }
    return ret;
}

/*******************************************************************************
 * Function Name: void low_power_post_sleep_cb(wiced_bool_t
 *                                                        restore_configuration)
 *******************************************************************************
 * Summary: Callback for post sleep initialization.
 *
 * Parameters:
 *   wiced_bool_t restore_configuration      : Tells whether the user needs to
 *                                              restore the configurations or not
 *
 * Return:
 *   None
 *
 ******************************************************************************/
void low_power_post_sleep_cb(wiced_bool_t restore_configuration)
{
    /* Not doing anything in this function right now. But it can be used to
       initialize peripherals such as I2C and SPI upon wakeup from ePDS */
    if(restore_configuration)
    {
        /* Add code to re-initialize peripherals */
    }
}'''

In [127]:
verbose=True
chain = doc_prompt | llm
ans = chain.invoke([]
    "query": '''Group: Home/Forums/WirelessConnectivity/AIROC Bluetooth
Topic: cybt-213043-02 not entering sleep mode
Hi,
I am using cybt-213043-02 eval board. I am running https://github.com/Infineon/mtb-example-btsdk-low-power-20819 example with a tweak that changes the eval board to cybt-213043-eval.
I can see the output on the PUART when I press the user button and the ble works ok.
However the board never enters sleep mode. I see a constant ~2.5mA usage and the sleep permit handler is never called. Also sleep callback is never called.
The HOST_WAKE and DEV_WAKE pins are floating. I have tried tying them to either VCC or GND without effect. Is there some additional configuration or hw requirement to trigger sleep mode.
Thank you for response.''',
    "context": low_power_as_context
    })

In [129]:
doc_prompt.format({})

TypeError: PromptTemplate.format() takes 1 positional argument but 2 were given

In [110]:
ans.content

'Based on the provided context, it seems that the example you are using is designed to demonstrate low power modes, including ePDS and HID-Off, with the CYW20819. However, you are experiencing issues with the CYBT-213043-02 eval board not entering sleep mode.\n\nHere are a few suggestions to troubleshoot and potentially resolve the issue:\n\n1. **Check Sleep Configuration**: Ensure that the sleep configuration is correctly set up in your code. The `low_power_sleep_config` structure should be properly initialized with the correct sleep mode, wake sources, and handlers.\n\n2. **Verify Button Configuration**: Make sure that the button you are using to trigger sleep is correctly configured in the code. The pin number for the button should match the one on your eval board.\n\n3. **Check Wake Pins**: The HOST_WAKE and DEV_WAKE pins should be configured correctly. If they are floating, it might cause issues. Ensure they are set up as per the requirements of your application.\n\n4. **Debugging

# check if pdf answers the question 

In [112]:
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader
os.environ['LLAMA_CLOUD_API_KEY'] = 'llx-bJs9Ppk0LMI1AIp1X4r04g7pBne4X1AH8DBMwwiVSrJCfPUb'
parser = LlamaParse(
    result_type="text"  # "markdown" and "text" are available
)

file_extractor = {".pdf": parser}
pdf_documents = SimpleDirectoryReader(input_files=['./Infineon-AN226546_CYW20819_Feature_and_Peripheral_Guide-ApplicationNotes-v04_00-EN.pdf'], file_extractor=file_extractor).load_data()
print(len(pdf_documents))

Started parsing the file under job_id a691c9f1-92c3-4fb9-83ae-233523b10ace
32


In [115]:
updated_docs_pdf = []

for document in pdf_documents:
    # Create a new Document instance with the updated attributes
    updated_document = Document(
        metadata=document.metadata,  # Keep metadata the same
        page_content=document.text,  # Change text to page_content
    )
    updated_docs_pdf.append(updated_document)

In [116]:
chunks_pdf = []
for doc in updated_docs_pdf:

    text_splitter = TokenTextSplitter(chunk_size=400, chunk_overlap=80)
    content = text_splitter.split_text(doc.page_content)
    for chunk in content:
        chunks_pdf.append(
            Document(
                metadata=doc.metadata,
                page_content = chunk
            ),
            
        )

In [117]:
uuids_pdf = [str(uuid4()) for doc in chunks_pdf]

In [ ]:
vector_store.add_documents(documents=chunks_pdf, ids=uuids_pdf)

In [212]:
results = vector_store.similarity_search(
    '''Group: Home/Forums/WirelessConnectivity/AIROC Bluetooth
Topic: cybt-213043-02 not entering sleep mode
Hi,
I am using cybt-213043-02 eval board. I am running https://github.com/Infineon/mtb-example-btsdk-low-power-20819 example with a tweak that changes the eval board to cybt-213043-eval.
I can see the output on the PUART when I press the user button and the ble works ok.
However the board never enters sleep mode. I see a constant ~2.5mA usage and the sleep permit handler is never called. Also sleep callback is never called.
The HOST_WAKE and DEV_WAKE pins are floating. I have tried tying them to either VCC or GND without effect. Is there some additional configuration or hw requirement to trigger sleep mode.
Thank you for response.''',
    k=5,
    filter={"repo":"mtb-example-btsdk-low-power-20819"}
)

In [213]:
[result.metadata for result in results]

[{'file_name': 'README.md',
  'file_path': 'README.md',
  'folder': '',
  'repo': 'mtb-example-btsdk-low-power-20819',
  'url': 'https://github.com/Infineon/mtb-example-btsdk-low-power-20819/blob/master/README.md'},
 {'file_name': 'low_power_20819.c',
  'file_path': 'low_power_20819.c',
  'folder': '',
  'repo': 'mtb-example-btsdk-low-power-20819',
  'url': 'https://github.com/Infineon/mtb-example-btsdk-low-power-20819/blob/master/low_power_20819.c'},
 {'file_name': 'low_power_20819_ble.c',
  'file_path': 'low_power_20819_ble.c',
  'folder': '',
  'repo': 'mtb-example-btsdk-low-power-20819',
  'url': 'https://github.com/Infineon/mtb-example-btsdk-low-power-20819/blob/master/low_power_20819_ble.c'},
 {'file_name': 'TARGET_CYW920820M2EVB-01.mtb',
  'file_path': 'deps/TARGET_CYW920820M2EVB-01.mtb',
  'folder': 'mtb-example-btsdk-low-power-20819/deps',
  'repo': 'mtb-example-btsdk-low-power-20819',
  'url': 'https://github.com/Infineon/mtb-example-btsdk-low-power-20819/blob/master/deps/TAR

In [123]:
for i in results:
    print(i[0].page_content)
    print()
    print()
    print()

infineon.com/cms/en/product/wireless-connectivity/airoc-bluetooth-le-bluetooth-multiprotocol/cyw20820/)

## Supported toolchains (make variable 'TOOLCHAIN')

- GNU Arm® Embedded Compiler v10.3.1 (`GCC_ARM`) - Default value of `TOOLCHAIN`

## Supported kits (make variable 'TARGET')
- [CYW920820M2EVB-01 evaluation kit](http://www.infineon.com/cyw920820m2evb-01) (`CYW920820M2EVB-01`) - Default value of `TARGET`
- [CYW920819M2EVB-01 evaluation kit](http://www.infineon.com/cyw920819m2evb-01) (`CYW920819M2EVB-01`)

## Hardware setup

This example uses the board's default configuration. See the kit user guide to ensure that the board is configured correctly.

## Software setup

This code example consists of two parts: a client and a server. For the client, download and install the LightBlue Connect App for [iOS](https://apps.apple.com/us/app/lightblue/id557428110) or [Android](https://play.google.com/store/apps/details?id=com.punchthrough.lightblueexplorer&gl=US).

Scan the following QR codes

In [120]:
chain = doc_prompt | llm
ans = chain.invoke({
    "query": '''Group: Home/Forums/WirelessConnectivity/AIROC Bluetooth
Topic: cybt-213043-02 not entering sleep mode
Hi,
I am using cybt-213043-02 eval board. I am running https://github.com/Infineon/mtb-example-btsdk-low-power-20819 example with a tweak that changes the eval board to cybt-213043-eval.
I can see the output on the PUART when I press the user button and the ble works ok.
However the board never enters sleep mode. I see a constant ~2.5mA usage and the sleep permit handler is never called. Also sleep callback is never called.
The HOST_WAKE and DEV_WAKE pins are floating. I have tried tying them to either VCC or GND without effect. Is there some additional configuration or hw requirement to trigger sleep mode.
Thank you for response.''',
    "context": results
    })

In [121]:
ans.content

"I don't have specific information about the CYBT-213043-02 evaluation board or its sleep mode configuration from the provided documents. However, based on the context of the CYW20819/CYW20820, here are a few general suggestions that might help:\n\n1. **Check GPIO Configuration**: Ensure that the GPIOs, especially HOST_WAKE and DEV_WAKE, are configured correctly. These pins are used for waking the Bluetooth device and the host device, respectively. Incorrect configuration might prevent the device from entering sleep mode.\n\n2. **Review Power Management Settings**: Verify that the power management settings in your application are correctly configured to allow the device to enter sleep mode. This might involve checking the sleep permit handler and ensuring it is set up to allow sleep.\n\n3. **Interrupts and Callbacks**: Make sure that any interrupts or callbacks related to sleep mode are correctly implemented and not inadvertently preventing sleep.\n\n4. **Hardware Connections**: Double

# Rough

In [135]:
import ast
import re
# Given string
response = "this is the answer: [1,2,3,4,5,6,7,8,9]"

# Use regex to find the list within brackets
match = re.search(r'\[(.*?)\]', response)

if match:
    # Extract the content inside the brackets
    list_string = match.group(0)  # This gets the whole match including brackets
    # Convert the string representation of the list to an actual list safely
    result_list = ast.literal_eval(list_string)
    print(result_list)  # Output: [1, 2, 3, 4, 5, 6, 7, 8, 9]
else:
    print("No list found")


[1, 2, 3, 4, 5, 6, 7, 8, 9]


In [146]:
vector_store.similarity_search("GIOP", filter={}, k=10)

[Document(id='146dadb3-a0df-4032-a3a5-2d1b2a4c7df7', metadata={'creation_date': '2024-10-12', 'file_name': 'Infineon-AN226546_CYW20819_Feature_and_Peripheral_Guide-ApplicationNotes-v04_00-EN.pdf', 'file_path': 'Infineon-AN226546_CYW20819_Feature_and_Peripheral_Guide-ApplicationNotes-v04_00-EN.pdf', 'file_size': 940024.0, 'file_type': 'application/pdf', 'last_modified_date': '2024-10-12'}, page_content='C\n                                                                                                                    2022-08-03'),
 Document(id='cc45effb-be4a-4478-9f17-4df9bb8026ce', metadata={'creation_date': '2024-10-12', 'file_name': 'Infineon-AN226546_CYW20819_Feature_and_Peripheral_Guide-ApplicationNotes-v04_00-EN.pdf', 'file_path': 'Infineon-AN226546_CYW20819_Feature_and_Peripheral_Guide-ApplicationNotes-v04_00-EN.pdf', 'file_size': 940024.0, 'file_type': 'application/pdf', 'last_modified_date': '2024-10-12'}, page_content='\n                                                     

In [153]:
for doc in documents:
    doc.metadata['repo']='mtb-example-btsdk-low-power-20819'
    doc.metadata['folder']=doc.metadata['file_path'].split('/')[0] if '/' in doc.metadata['file_path'] else ''
    print(doc.metadata)

{'file_path': '.gitignore', 'file_name': '.gitignore', 'url': 'https://github.com/Infineon/mtb-example-btsdk-low-power-20819/blob/master/.gitignore', 'repo': 'mtb-example-btsdk-low-power-20819', 'folder': ''}
{'file_path': 'LICENSE', 'file_name': 'LICENSE', 'url': 'https://github.com/Infineon/mtb-example-btsdk-low-power-20819/blob/master/LICENSE', 'repo': 'mtb-example-btsdk-low-power-20819', 'folder': ''}
{'file_path': 'Makefile', 'file_name': 'Makefile', 'url': 'https://github.com/Infineon/mtb-example-btsdk-low-power-20819/blob/master/Makefile', 'repo': 'mtb-example-btsdk-low-power-20819', 'folder': ''}
{'file_path': 'README.md', 'file_name': 'README.md', 'url': 'https://github.com/Infineon/mtb-example-btsdk-low-power-20819/blob/master/README.md', 'repo': 'mtb-example-btsdk-low-power-20819', 'folder': ''}
{'file_path': 'app_bt_cfg.c', 'file_name': 'app_bt_cfg.c', 'url': 'https://github.com/Infineon/mtb-example-btsdk-low-power-20819/blob/master/app_bt_cfg.c', 'repo': 'mtb-example-btsdk

In [173]:
a=[1,2,3]
a.append(4)

In [179]:
a = vector_store.similarity_search_with_relevance_scores("hi how are you")
for i in a:
    print(i[1])

0.8565048875
0.8558888735
0.8557799755
0.85563311


In [190]:
from collections import Counter
repos = ['abc','abc','def','def','def']
occurences = Counter(repos)
occurences.most_common()[0][0]

'def'

In [195]:
occ = Counter({'mtb-example-btsdk-low-power-20819': 5})
rc = occ.most_common()[0][0]
rc

'mtb-example-btsdk-low-power-20819'

In [196]:
a=['','','','']
a=Counter(a)
a.most_common()

[('', 4)]

In [208]:
a = ["Infineon-AN226546_CYW20819_Feature_and_Peripheral_Guide-ApplicationNotes-v04_00-EN", "Infineon-AN226546_CYW20819_Feature_and_Peripheral_Guide-ApplicationNotes-v04_00-EN"]

b=['./' + filename + '.pdf' for filename in a]
b

['./Infineon-AN226546_CYW20819_Feature_and_Peripheral_Guide-ApplicationNotes-v04_00-EN.pdf',
 './Infineon-AN226546_CYW20819_Feature_and_Peripheral_Guide-ApplicationNotes-v04_00-EN.pdf']

In [230]:
links = ['https://github.com/Infineon/mtb-example-btsdk-low-power-20819/blob/master/README.md', 'https://github.com/Infineon/mtb-example-btsdk-low-power-20819/blob/master/README.md', 'https://github.com/Infineon/mtb-example-btsdk-low-power-20819/blob/master/README.md', 'https://github.com/Infineon/mtb-example-btsdk-low-power-20819/blob/master/README.md', 'https://github.com/Infineon/mtb-example-btsdk-low-power-20819/blob/master/low_power_20819_ble.c']
l = [link.replace("https:/github.com/","https://raw.githubusercontent.com/") for link in links]

link_md = ["https://raw.githubusercontent.com/Infineon/mtb-example-btsdk-low-power-20819/refs/heads/master/low_power_20819_ble.c"]

link_from_agent=["https://github.com/Infineon/mtb-example-btsdk-low-power-20819/blob/master/low_power_20819_ble.c"]


In [ ]:
import requests

# # List of GitHub raw URLs
# urls = [
#     "https://raw.githubusercontent.com/user/repository/main/file1.md",
#     "https://raw.githubusercontent.com/user/repository/main/file2.c"
#     # Add more URLs as needed
# ]

def fetch_file_content(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        return f"Failed to retrieve {url}, status code: {response.status_code}"

# Fetch and print content of each file
# for url in l:
content = fetch_file_content(link_from_agent[0].replace("https://github","https://raw.githubusercontent"))
print(f"Content from {link_from_agent[0]}:\n{content}\n")


In [231]:
link_from_agent[0].replace("https://github","https://raw.githubusercontent").replace("blob","refs/heads") == link_md[0]

True